# Fase 2: Limpieza de Datos

In [1]:
import pandas as pd
import numpy as np
import json

import sys
import os

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import soporte_limpieza as sl

In [2]:
df = pd.read_csv("../datos/tablas_concatenadas.csv")
df.head(2)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
0,0,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",0.0,0.0,0.0,0.0,2013-12-31,2013.0
1,1,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,0.0,0.0,2013-12-31,2013.0


In [3]:
df.isnull().sum()

Unnamed: 0                       0
id organizacion superior     30351
organizacion superior       357007
id organizacion              25111
organizacion                 34861
id unidad gestora            33566
unidad gestora               19470
categoria economica          18971
origen ingreso               38394
tipo ingreso                 31905
valor previsto               51276
valor registrado             51276
valor recaudado              51276
porcentaje recaudado         51276
fecha recaudacion            16324
anio recaudacion                 0
dtype: int64

### 1- Gestión de los nulos de id organizacion superior y organizacion superior.

Vamos a ver si podemos crear un diccionario para los id y los nombres de la organización superior.

In [4]:
len(df["id organizacion superior"].unique())

26

In [5]:
len(df["organizacion superior"].unique())

26

A priori parece que hay el mismo numero de ids que de nombres, buena señal.

In [6]:
lista_id_os = list(df["id organizacion superior"].unique())
lista_id_os

[np.float64(63000.0),
 np.float64(37000.0),
 np.float64(nan),
 np.float64(22000.0),
 np.float64(55000.0),
 np.float64(24000.0),
 np.float64(52000.0),
 np.float64(25000.0),
 np.float64(26000.0),
 np.float64(39000.0),
 np.float64(30000.0),
 np.float64(81000.0),
 np.float64(58000.0),
 np.float64(33000.0),
 np.float64(36000.0),
 np.float64(41000.0),
 np.float64(57000.0),
 np.float64(35000.0),
 np.float64(32000.0),
 np.float64(49000.0),
 np.float64(53000.0),
 np.float64(51000.0),
 np.float64(44000.0),
 np.float64(38000.0),
 np.float64(54000.0),
 np.float64(20000.0)]

In [7]:
for id in lista_id_os:
    print(f"\n{id}")
    filtro1 = df["id organizacion superior"] == float(id)
    filtro2 = df["organizacion superior"].notnull()
    print(len(df[filtro1 & filtro2]["organizacion superior"].unique()))


63000.0
1

37000.0
1

nan
0

22000.0
1

55000.0
1

24000.0
1

52000.0
1

25000.0
1

26000.0
1

39000.0
1

30000.0
1

81000.0
1

58000.0
1

33000.0
1

36000.0
1

41000.0
1

57000.0
1

35000.0
1

32000.0
1

49000.0
1

53000.0
1

51000.0
1

44000.0
1

38000.0
1

54000.0
1

20000.0
1


Como vemos que para cada id hay un unico valor creamos un diccionario:

In [8]:
dic_id_os = {}
for id in lista_id_os:
    try:
        filtro1 = df["id organizacion superior"] == float(id)
        filtro2 = df["organizacion superior"].notnull()
        dic_id_os[float(id)]=df[filtro1 & filtro2]["organizacion superior"].unique()[0]
    except:
        pass

In [9]:
dic_id_os

{63000.0: 'Advocacia-Geral da União',
 37000.0: 'Controladoria-Geral da União',
 22000.0: 'Ministério da Agricultura, Pecuária e Abastec',
 55000.0: 'Ministério da Cidadania',
 24000.0: 'Ministério da Ciência, Tecnologia, Inovações ',
 52000.0: 'Ministério da Defesa',
 25000.0: 'Ministério da Economia',
 26000.0: 'Ministério da Educação',
 39000.0: 'Ministério da Infraestrutura',
 30000.0: 'Ministério da Justiça e Segurança Pública',
 81000.0: 'Ministério da Mulher, Família e Direitos Huma',
 58000.0: 'Ministério da Pesca e Aquicultura',
 33000.0: 'Ministério da Previdência Social',
 36000.0: 'Ministério da Saúde',
 41000.0: 'Ministério das Comunicações',
 57000.0: 'Ministério das Mulheres, Igualdade Racial, da',
 35000.0: 'Ministério das Relações Exteriores',
 32000.0: 'Ministério de Minas e Energia',
 49000.0: 'Ministério do Desenvolvimento Agrário',
 53000.0: 'Ministério do Desenvolvimento Regional',
 51000.0: 'Ministério do Esporte',
 44000.0: 'Ministério do Meio Ambiente',
 38000.

Ya con nuestro diccionario vamos a rellenar los nombres e id de las organizaciones superiores.

Hemos comprobado que todos los campos de organización superior concuerdan con su id a excepción de cuando son NaN, corregimos los NaN. además quedarán sin corregir aquellas filas donde id y organización superioir son ambos NaN.

In [10]:
# filas que van a quedar sin corregir.
filtro1 = df["id organizacion superior"].isnull()
filtro2 = df["organizacion superior"].isnull()
df[filtro1 & filtro2].shape

(10543, 16)

In [11]:
df["organizacion superior"].fillna(df["id organizacion superior"].map(dic_id_os), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_23924\3684483159.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["organizacion superior"].fillna(df["id organizacion superior"].map(dic_id_os), inplace=True)


In [12]:
os_null = df[df["organizacion superior"].isnull()].shape[0]
os_sin_corregir = os_null/df.shape[0]*100
print(f"Han quedado a null {os_null}, es decir, un {np.round(os_sin_corregir,2)}% del total de las organizaciones superiores.")

Han quedado a null 10543, es decir, un 1.04% del total de las organizaciones superiores.


In [13]:
df[df["id organizacion superior"].isnull()].shape[0]

30351

Como podemos ver ids hay 30351 a null los cuales podemos corregir al rededor de unos 20000 a partir del nombre de la organizacion superior.

In [14]:
#Invertimos el diccionario
dic_organizaciones_id = {valor: clave for clave, valor in dic_id_os.items()}
dic_organizaciones_id


{'Advocacia-Geral da União': 63000.0,
 'Controladoria-Geral da União': 37000.0,
 'Ministério da Agricultura, Pecuária e Abastec': 22000.0,
 'Ministério da Cidadania': 55000.0,
 'Ministério da Ciência, Tecnologia, Inovações ': 24000.0,
 'Ministério da Defesa': 52000.0,
 'Ministério da Economia': 25000.0,
 'Ministério da Educação': 26000.0,
 'Ministério da Infraestrutura': 39000.0,
 'Ministério da Justiça e Segurança Pública': 30000.0,
 'Ministério da Mulher, Família e Direitos Huma': 81000.0,
 'Ministério da Pesca e Aquicultura': 58000.0,
 'Ministério da Previdência Social': 33000.0,
 'Ministério da Saúde': 36000.0,
 'Ministério das Comunicações': 41000.0,
 'Ministério das Mulheres, Igualdade Racial, da': 57000.0,
 'Ministério das Relações Exteriores': 35000.0,
 'Ministério de Minas e Energia': 32000.0,
 'Ministério do Desenvolvimento Agrário': 49000.0,
 'Ministério do Desenvolvimento Regional': 53000.0,
 'Ministério do Esporte': 51000.0,
 'Ministério do Meio Ambiente': 44000.0,
 'Minis

In [15]:
df["id organizacion superior"].fillna(df["organizacion superior"].map(dic_organizaciones_id), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_23924\1809695679.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["id organizacion superior"].fillna(df["organizacion superior"].map(dic_organizaciones_id), inplace=True)


In [16]:
df[df["id organizacion superior"].isnull()].shape[0]

10543

Comprobamos cuantos null hay ahora y efectivamente se han reducido hasta 10543, los mismo que para los nombres de las organizaciones superiores ya que son todas las filas que tienen ambos campos a null. Aunque al represenatr un 1% y al no haber forma de arreglarlo puede ser despreciable.

### 2- Gestión de los nulos de id organizacion y organizacion.

Vamos a seguir la misma estrategia que en el paso anterior.

In [17]:
len(df["id organizacion"].unique())

292

In [18]:
len(df["organizacion"].unique())

288

Hay más ids que organizaciones, lo cual impica que una misma organización puede tener 2 id, veámoslo.

In [19]:
lista_organizaciones = list(df["organizacion"].unique())
len(lista_organizaciones)

288

In [20]:
for orga in lista_organizaciones:
    filtro1 = df["id organizacion"].notnull()
    filtro2 = df["organizacion"] == orga
    if len(df[filtro1 & filtro2]["id organizacion"].unique()) > 1:
        print(f"\n{orga}")
        print(df[filtro1 & filtro2]["id organizacion"].unique())


Agência Nacional de Transportes Aquaviários
[68201. 39251.]

Fundo Nacional Antidrogas
[30912. 20117.]

Fundo Nacional do Idoso
[64902. 30914.]

Fundo Nacional para a Criança e o Adolescente
[64901. 30913.]


Efectivamente vemos que hay tres organizaciones con dos id. Vamos a ver que está pasando.

- Para la Agência Nacional de Transportes Aquaviários vemos que el id 39251 solo seusa en 5 filas y todas son de 2013 por lo que puede ser que sea un id viejo y haya transitado hacia el nuevo que es 68201, por lo que vamos a cambiar el id viejo por el nuevo para homogeneizar los datos.

In [21]:
filtro1_1 = df["id organizacion"] == 68201
df[filtro1_1].shape
df[filtro1_1].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3304,3304,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,0.0,0.0,2013-12-31,2013.0
3356,3356,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2013-12-31,2013.0
3366,3366,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Receita de Serviços,Receita de Serviços,0.0,0.0,0.0,0.0,2013-12-31,2013.0
3401,3401,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2013-12-31,2013.0
7843,7844,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",NaN,NaN,NaN,NaN,2014-12-31,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985803,993258,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2021-07-27,2021.0
985804,993259,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2021-01-22,2021.0
985805,993260,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2021-07-09,2021.0
985806,993261,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2021-09-16,2021.0


In [42]:
filtro1_2 = df["id organizacion"] == 39251
df[filtro1_2].shape
df[filtro1_2].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion


In [23]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_39251)

In [24]:
filtro1 = df["id organizacion"] == 39251
df[filtro1].shape

(0, 16)

- Para la Fundo Nacional Antidrogas pasa algo similar, sin embargo, el id 20117 aparece 20 veces y de forma aleatoria pues aparece en 2013, 2014, 2015, 2016 y hasta 2017, pero no a partir de 2017, lo que puede indicar que se fue transicionando de id durante varios años usandose dos a la vez hasta que en 2017 se dejó de usar. Además también se observa que para ambos ids a pesar de tener la misma unidad gestora también tienen ids de unidad gestora distintos.

IMPORTANTE:

He vuelto sobre mis pasos y me he dado cuenta de que la organización superior para los distintos id es distinta, por lo que si los unificara todos es como si estuviera perdiendo información ya que aunque la organización sea la misma si tiene el id de organización 30912 quiere decir que su organización superior es Ministério da Justiça e Segurança Pública y si tiene el id 20117 su organización superioir es Presidência da República. Me he dado cuenta de que pasa lo mismo también para los dos siguientes. En mi opinión unificar ambos id y dejar el que más aparece implicaría en cierta forma perder información, ya que si yo tengo un valor NaN para orga superioir e id de orga superior pero con id de organización 20117 se puede inferir que la organización superior es Presidência da República. Pero si unificasemos los id y pusiesemos para la organización Fundo Nacional Antidroga el id 30912 si tenemos un valor en NaN para id de orga superior y orga superior no podemos inferir si era Presidência da República o Ministério da Justiça e Segurança Pública. Pero como hemos visto que solo tenemos a NaN un 1% de las filas esto lo gestionaré si tuviera tiempo más adelante.

In [25]:
filtro2_1 = df["id organizacion"] == 30912
print(df[filtro2_1].shape)
print(df[filtro2_1]["id unidad gestora"].unique())
print(df[filtro2_1]["unidad gestora"].unique())
df[filtro2_1].head(1)

(7322, 16)
[200246.     nan]
['FUNDO NACIONAL ANTIDROGAS' nan]


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3447,3447,30000.0,Ministério da Justiça e Segurança Pública,30912.0,Fundo Nacional Antidrogas,200246.0,FUNDO NACIONAL ANTIDROGAS,Receitas Correntes,NaN,"Multas administrativas, contratuais e judicia",0.0,0.0,0.0,0.0,2013-12-31,2013.0


In [26]:
filtro2_2 = df["id organizacion"] == 20117
print(df[filtro2_2].shape)
print(df[filtro2_2]["id unidad gestora"].unique())
print(df[filtro2_2]["unidad gestora"].unique())
df[filtro2_2].head(1)

(20, 16)
[110246.]
['FUNDO NACIONAL ANTIDROGAS']


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
4481,4481,20000.0,Presidência da República,20117.0,Fundo Nacional Antidrogas,110246.0,FUNDO NACIONAL ANTIDROGAS,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,0.0,0.0,0.0,0.0,2013-12-31,2013.0


In [27]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_20117)

In [28]:
filtro2 = df["id organizacion"] == 20117
df[filtro2].shape

(0, 16)

- Continuamos con los id de Fundo Nacional do Idoso. De nuevo ocurre lo mismo, el id 64902 aparece solo 99 veces y hasta 2017, el id 30914 aparece 223 y a partir de 2017 lo que indica un camio de id, siendo el actual 30914. De nuevo los id de unidad gestora son distintos aunque la unidad gestora sea la misma.


In [41]:
filtro3_1 = df["id organizacion"] == 64902
print(df[filtro3_1].shape)
df[filtro3_1].head(1)

(0, 16)


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion


In [40]:
filtro3_2 = df["id organizacion"] == 30914
print(df[filtro3_2].shape)
df[filtro3_2].head(1)

(322, 16)


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3784,3784,57000.0,"Ministério das Mulheres, Igualdade Racial, da",30914.0,Fundo Nacional do Idoso,207001.0,FUNDO NACIONAL DO IDOSO,Receitas Correntes,Transferências Correntes,Transferências dos Municípios e de suas Entid,3000000.0,3000000.0,3000000.0,3000000.0,2013-12-31,2013.0


In [31]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_64902)

In [32]:
filtro3 = df["id organizacion"] == 64902
df[filtro2].shape

(0, 16)

- Terminamos con Fundo Nacional para a Criança e o Adolescente, en este caso ambos id se usn en 2021, el 64901 se empieza a usar en 2013 mientras que es 30913 se usa a apartir de 2017, por lo que vamos a dejar 30913.

In [33]:
filtro4_1 = df["id organizacion"] == 64901
print(df[filtro4_1].shape)
filtro4_2 = df["id organizacion"] == 30913
print(df[filtro4_2].shape)

(180, 16)
(719, 16)


In [39]:
df[filtro4_2].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
356944,359789,81000.0,"Ministério da Mulher, Família e Direitos Huma",30913.0,Fundo Nacional para a Criança e o Adolescente,307001.0,FUNDO NACIONAL PARA A CRIANCA E O ADOLESCENTE,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,0.0,0.0,0.0,0.0,2017-06-20,2017.0


In [38]:
df[filtro4_1].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3785,3785,57000.0,"Ministério das Mulheres, Igualdade Racial, da",30913.0,Fundo Nacional para a Criança e o Adolescente,110244.0,FUNDO NACIONAL PARA A CRIANCA E O ADOLESCENTE,Receitas Correntes,Transferências Correntes,Transferências dos Municípios e de suas Entid,24290692.0,24290692.0,24290692.0,24290692.0,2013-12-31,2013.0


In [36]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_64901)

In [37]:
filtro4 = df["id organizacion"] == 64901
df[filtro4].shape

(0, 16)

In [45]:
#Comprobamos que ya solo hay un id por organización
flag = 0
for orga in lista_organizaciones:
    filtro1 = df["id organizacion"].notnull()
    filtro2 = df["organizacion"] == orga
    if len(df[filtro1 & filtro2]["id organizacion"].unique()) > 1:
        flag=1
        print(f"\n{orga}")
        print(df[filtro1 & filtro2]["id organizacion"].unique())
print(flag)

0


Ya podemos hacer un diccionario para rellenar valores.

In [50]:
lista_id_organizaciones = list(df["id organizacion"].unique())
print(len(lista_id_organizaciones))

288


In [51]:
dic_id_organizacion = {}
for id in lista_id_organizaciones:
    try:
        filtro1 = df["id organizacion"] == float(id)
        filtro2 = df["organizacion"].notnull()
        dic_id_organizacion[float(id)]=df[filtro1 & filtro2]["organizacion"].unique()[0]
    except:
        pass

In [54]:
df["id organizacion"].isnull().sum()

np.int64(25111)

In [53]:
# filas que van a quedar sin corregir.
filtro1 = df["id organizacion"].isnull()
filtro2 = df["organizacion"].isnull()
df[filtro1 & filtro2].shape

(905, 16)

In [55]:
df["organizacion"].fillna(df["id organizacion"].map(dic_id_organizacion), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_23924\3881576057.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["organizacion"].fillna(df["id organizacion"].map(dic_id_organizacion), inplace=True)


In [56]:
orga_null = df[df["organizacion"].isnull()].shape[0]
orga_sin_corregir = orga_null/df.shape[0]*100
print(f"Han quedado a null {orga_null}, es decir, un {np.round(orga_sin_corregir,2)}% del total de las organizaciones.")

Han quedado a null 905, es decir, un 0.09% del total de las organizaciones.


In [57]:
df[df["id organizacion"].isnull()].shape[0]

25111

Como podemos ver ids hay 25111 a null los cuales podemos corregir al rededor de unos 24200 a partir del nombre de la organizacion.

In [59]:
#Invertimos el diccionario
dic_orga_id = {valor: clave for clave, valor in dic_id_organizacion.items()}

In [60]:
df["id organizacion"].fillna(df["organizacion"].map(dic_orga_id), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_23924\2430629590.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["id organizacion"].fillna(df["organizacion"].map(dic_orga_id), inplace=True)


In [61]:
df[df["id organizacion"].isnull()].shape[0]

905

Comprobamos cuantos null hay ahora y efectivamente se han reducido hasta 905, los mismo que para los nombres de las organizaciones ya que son todas las filas que tienen ambos campos a null. Aunque al representar un 0.1% y al no haber forma de arreglarlo puede ser despreciable.

# Conclusiones

1- Hemos corregido los nombres de las organizaciones superiores a partir de los id y han quedado solo 10543 sin corregir lo que representa un 1.04% del total. Lo mismo hemos hecho para corregir los id a partir del nombre de la organización superioir, de nuevo queda un 1.04% sin corregir el cual representa las filas donde ambas columnas tienen valores nulos.

2- Hemos corregido los nombres de las organizaciones a partir de los id y han quedado solo 905 sin corregir lo que representa un 0.1% del total. Lo mismo hemos hecho para corregir los id a partir del nombre de la organización, de nuevo queda un 0.1% sin corregir el cual representa las filas donde ambas columnas tienen valores nulos.